In [1]:
import os
os.environ['PYSPARK_PYTHON'] = '/opt/conda/envs/python2/bin/python'
import pyspark
sc = pyspark.SparkContext("local")

# do something to prove it works
#rdd = sc.parallelize(range(1000))
#rdd.takeSample(False, 5)

In [15]:
#Formatear un CSV a DataFrame

#Librerias necesarias
from pyspark.sql.types import *
from pyspark.sql import Row, SQLContext

#Cargar de un archivo csv un Resilient Distributed Dataset
rdd = sc.textFile('Sacramentorealestatetransactions.csv')
hasattr(rdd, "toDF")

sqlContext = SQLContext(sc) # or HiveContext
hasattr(rdd, "toDF")

True

In [16]:
#Muestra las primeras 5 lineas
rdd.take(5)

#Separa por coma cada linea. Operacion LAZY
rdd = rdd.map(lambda line: line.split(","))

In [17]:
#Muestra los primeros 2 vectores
rdd.take(2)

[[u'street',
  u'city',
  u'zip',
  u'state',
  u'beds',
  u'baths',
  u'sq__ft',
  u'type',
  u'sale_date',
  u'price',
  u'latitude',
  u'longitude'],
 [u'3526 HIGH ST',
  u'SACRAMENTO',
  u'95838',
  u'CA',
  u'2',
  u'1',
  u'836',
  u'Residential',
  u'Wed May 21 00:00:00 EDT 2008',
  u'59222',
  u'38.631913',
  u'-121.434879']]

In [18]:
#Se guarda la primera linea del archivo (Descripcion de los campos)
first_line=rdd.first()

#Filtra la primera linea del rdd
rdd = rdd.filter(lambda line:line != first_line)

#Genera un DataFrame  y nombre las distintas columnas
df = rdd.map(lambda line: Row(street = line[0], city = line[1], zip=line[2], beds=int(line[4]), baths=int(line[5]), sqft=int(line[6]), price=int(line[9]))).toDF()

#Muestra el DataFrame
df.show()

+-----+----+--------------+------+----+--------------------+-----+
|baths|beds|          city| price|sqft|              street|  zip|
+-----+----+--------------+------+----+--------------------+-----+
|    1|   2|    SACRAMENTO| 59222| 836|        3526 HIGH ST|95838|
|    1|   3|    SACRAMENTO| 68212|1167|         51 OMAHA CT|95823|
|    1|   2|    SACRAMENTO| 68880| 796|      2796 BRANCH ST|95815|
|    1|   2|    SACRAMENTO| 69307| 852|    2805 JANETTE WAY|95815|
|    1|   2|    SACRAMENTO| 81900| 797|     6001 MCMAHON DR|95824|
|    1|   3|    SACRAMENTO| 89921|1122|  5828 PEPPERMILL CT|95841|
|    2|   3|    SACRAMENTO| 90895|1104| 6048 OGDEN NASH WAY|95842|
|    1|   3|    SACRAMENTO| 91002|1177|       2561 19TH AVE|95820|
|    2|   2|RANCHO CORDOVA| 94905| 941|11150 TRINITY RIV...|95670|
|    2|   3|     RIO LINDA| 98937|1146|        7325 10TH ST|95673|
|    2|   3|    SACRAMENTO|100309| 909|    645 MORRISON AVE|95838|
|    2|   3|    SACRAMENTO|106250|1289|       4085 FAWN CIR|95

In [19]:
#Se puede transformar a Libreria Pandas
df.toPandas().head()





,baths,beds,city,price,sqft,street,zip
0,1,2,SACRAMENTO,59222,836,3526 HIGH ST,95838
1,1,3,SACRAMENTO,68212,1167,51 OMAHA CT,95823
2,1,2,SACRAMENTO,68880,796,2796 BRANCH ST,95815
3,1,2,SACRAMENTO,69307,852,2805 JANETTE WAY,95815
4,1,2,SACRAMENTO,81900,797,6001 MCMAHON DR,95824


In [20]:
#Se genera una sub_tabla donde se filtra por zip
favorite_zip = df[df.zip == 95815]

#Se muestran las primeras 5 entradas
favorite_zip.show(5)



+-----+----+----------+------+----+----------------+-----+
|baths|beds|      city| price|sqft|          street|  zip|
+-----+----+----------+------+----+----------------+-----+
|    1|   2|SACRAMENTO| 68880| 796|  2796 BRANCH ST|95815|
|    1|   2|SACRAMENTO| 69307| 852|2805 JANETTE WAY|95815|
|    1|   1|SACRAMENTO|106852| 871| 2930 LA ROSA RD|95815|
|    1|   2|SACRAMENTO| 78000| 800|    3132 CLAY ST|95815|
|    2|   4|SACRAMENTO| 89000|1316| 483 ARCADE BLVD|95815|
+-----+----+----------+------+----+----------------+-----+
only showing top 5 rows



In [21]:
#Se seleccionan esas 2 columnas y se muestran las primeras 10
df.select('city','beds').show(10)



+--------------+----+
|          city|beds|
+--------------+----+
|    SACRAMENTO|   2|
|    SACRAMENTO|   3|
|    SACRAMENTO|   2|
|    SACRAMENTO|   2|
|    SACRAMENTO|   2|
|    SACRAMENTO|   3|
|    SACRAMENTO|   3|
|    SACRAMENTO|   3|
|RANCHO CORDOVA|   2|
|     RIO LINDA|   3|
+--------------+----+
only showing top 10 rows



In [22]:
#Se agrupa por cama y se cuenta cuantas entradas hay por grupo.
df.groupBy("beds").count().show()



+----+-----+
|beds|count|
+----+-----+
|   0|  108|
|   6|    3|
|   5|   59|
|   1|   10|
|   3|  413|
|   8|    1|
|   2|  133|
|   4|  258|
+----+-----+



In [23]:
#Se muestran algunos indicacores estadisticos de esos 4 campos
df.describe(['baths', 'beds','price','sqft']).show()

+-------+------------------+------------------+------------------+------------------+
|summary|             baths|              beds|             price|              sqft|
+-------+------------------+------------------+------------------+------------------+
|  count|               985|               985|               985|               985|
|   mean|1.7766497461928934|2.9116751269035532|234144.26395939087|1314.9167512690356|
| stddev|0.8953714223186463|1.3079322320435807|138365.83908492787| 853.0482425034448|
|    min|                 0|                 0|              1551|                 0|
|    max|                 5|                 8|            884790|              5822|
+-------+------------------+------------------+------------------+------------------+



In [26]:
#REGRESSION


#Importar modulos necesarios
import pyspark.mllib
import pyspark.mllib.regression
from pyspark.mllib.regression import LabeledPoint
from pyspark.sql.functions import *

#Crear un DataFrame a partir de las variables que interesan. Pronosticar el precio, por medio de la cantidad de baños, camas, y pies cuadrados.
df = df.select('price','baths','beds','sqft')

#Muestra informacion de las variables
df.describe(['baths','beds','price','sqft']).show()

+-------+------------------+------------------+------------------+------------------+
|summary|             baths|              beds|             price|              sqft|
+-------+------------------+------------------+------------------+------------------+
|  count|               814|               814|               814|               814|
|   mean|1.9606879606879606|3.2444717444717446| 229448.3697788698|1591.1461916461917|
| stddev|0.6698038253879438|0.8521372615281972|119825.57606009027| 663.8419297942895|
|    min|                 1|                 1|              2000|               484|
|    max|                 5|                 8|            884790|              5822|
+-------+------------------+------------------+------------------+------------------+



In [27]:
#Filtra ejemplos cuyas variables sean mayor a 0
df = df[df.baths > 0]
df = df[df.beds > 0]
df = df[df.sqft > 0]

#Muestra informacion de las variables
df.describe(['baths','beds','price','sqft']).show()




+-------+------------------+------------------+------------------+------------------+
|summary|             baths|              beds|             price|              sqft|
+-------+------------------+------------------+------------------+------------------+
|  count|               814|               814|               814|               814|
|   mean|1.9606879606879606|3.2444717444717446| 229448.3697788698|1591.1461916461917|
| stddev|0.6698038253879438|0.8521372615281972|119825.57606009027| 663.8419297942895|
|    min|                 1|                 1|              2000|               484|
|    max|                 5|                 8|            884790|              5822|
+-------+------------------+------------------+------------------+------------------+



In [28]:
#Crea un RDD de LabeledPoints
temp = df.rdd.map(lambda line:LabeledPoint(line[0],[line[1:]]))

#La variable Sqft es de una escala mucho mayor a las otras 2 variables
temp.take(5)

[LabeledPoint(59222.0, [1.0,2.0,836.0]),
 LabeledPoint(68212.0, [1.0,3.0,1167.0]),
 LabeledPoint(68880.0, [1.0,2.0,796.0]),
 LabeledPoint(69307.0, [1.0,2.0,852.0]),
 LabeledPoint(81900.0, [1.0,2.0,797.0])]

In [29]:
#Importar modulos necesarios
from pyspark.mllib.util import MLUtils
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.feature import StandardScaler

#RDD de solo las variables
features = df.rdd.map(lambda row: row[1:])
features.take(5)


[(1, 2, 836), (1, 3, 1167), (1, 2, 796), (1, 2, 852), (1, 2, 797)]

In [30]:
#Escala todas las variables.
standardizer = StandardScaler()
model = standardizer.fit(features)
features_transform = model.transform(features)
features_transform.take(5)

#Se genera un RDD con solo las etiquetas
lab = df.rdd.map(lambda row: row[0])
lab.take(5)

#Se une la etiqueta con las variables transformadas
transformedData = lab.zip(features_transform)
transformedData.take(5)

[(59222, DenseVector([1.493, 2.347, 1.2593])),
 (68212, DenseVector([1.493, 3.5206, 1.7579])),
 (68880, DenseVector([1.493, 2.347, 1.1991])),
 (69307, DenseVector([1.493, 2.347, 1.2834])),
 (81900, DenseVector([1.493, 2.347, 1.2006]))]

In [31]:
#Se genera un RDD de LabeledPoints a partir de transformedData
transformedData = transformedData.map(lambda row: LabeledPoint(row[0],[row[1]]))
transformedData.take(5)

#Se separa en Training y Testing
trainingData, testingData = transformedData.randomSplit([.8,.2],seed=1234)

#Se genera una Regresion lineal con Stochastic Gradient Descent
from pyspark.mllib.regression import LinearRegressionWithSGD
linearModel = LinearRegressionWithSGD.train(trainingData,1000,.2)

#Muestra los pesos del modelo
linearModel.weights

#Muestra los 10 primeros ejemplos de la data de test
testingData.take(10)

#Se usa el modelo para predecir la etiqueta de este ejemplo
linearModel.predict([1.49297445326,3.52055958053,1.73535287287])

/usr/local/spark/python/pyspark/mllib/regression.py:281: UserWarning: Deprecated in 2.0.0. Use ml.regression.LinearRegression.
  warnings.warn("Deprecated in 2.0.0. Use ml.regression.LinearRegression.")


161670.25323743909

In [32]:
#Se usan algunas metricas para chequear el modelo

from pyspark.mllib.evaluation import RegressionMetrics

#Se genera un RDD con la salida predicha y la etiqueta real (valor real) Datos de Entrenamiento
prediObserRDDin = trainingData.map(lambda row: (float(linearModel.predict(row.features[0])),row.label))

metrics = RegressionMetrics(prediObserRDDin)
metrics.r2

#Resultados Datos de Test
prediObserRDDout = testingData.map(lambda row: (float(linearModel.predict(row.features[0])),row.label))
metrics = RegressionMetrics(prediObserRDDout)

#Se muestra el error cuadratico medio
metrics.rootMeanSquaredError

84809.67282565947